# Language Translation
In this project, you’re going to take a peek into the realm of neural network machine translation.  You’ll be training a sequence to sequence model on a dataset of English and French sentences that can translate new sentences from English to French.
## Get the Data
Since translating the whole language of English to French will take lots of time to train, we have provided you with a small portion of the English corpus.

In [1]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper
import problem_unittests as tests

source_path = 'data/small_vocab_en'
target_path = 'data/small_vocab_fr'
source_text = helper.load_data(source_path)
target_text = helper.load_data(target_path)

## Explore the Data
Play around with view_sentence_range to view different parts of the data.

In [2]:
view_sentence_range = (0, 10)

"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import numpy as np

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in source_text.split()})))

sentences = source_text.split('\n')
word_counts = [len(sentence.split()) for sentence in sentences]
print('Number of sentences: {}'.format(len(sentences)))
print('Average number of words in a sentence: {}'.format(np.average(word_counts)))

print()
print('English sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(source_text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))
print()
print('French sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(target_text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))

Dataset Stats
Roughly the number of unique words: 227
Number of sentences: 137861
Average number of words in a sentence: 13.225277634719028

English sentences 0 to 10:
new jersey is sometimes quiet during autumn , and it is snowy in april .
the united states is usually chilly during july , and it is usually freezing in november .
california is usually quiet during march , and it is usually hot in june .
the united states is sometimes mild during june , and it is cold in september .
your least liked fruit is the grape , but my least liked is the apple .
his favorite fruit is the orange , but my favorite is the grape .
paris is relaxing during december , but it is usually chilly in july .
new jersey is busy during spring , and it is never hot in march .
our least liked fruit is the lemon , but my least liked is the grape .
the united states is sometimes busy during january , and it is sometimes warm in november .

French sentences 0 to 10:
new jersey est parfois calme pendant l' automne 

## Implement Preprocessing Function
### Text to Word Ids
As you did with other RNNs, you must turn the text into a number so the computer can understand it. In the function `text_to_ids()`, you'll turn `source_text` and `target_text` from words to ids.  However, you need to add the `<EOS>` word id at the end of `target_text`.  This will help the neural network predict when the sentence should end.

You can get the `<EOS>` word id by doing:
```python
target_vocab_to_int['<EOS>']
```
You can get other word ids using `source_vocab_to_int` and `target_vocab_to_int`.

In [3]:
def text_to_ids(source_text, target_text, source_vocab_to_int, target_vocab_to_int):
    """
    Convert source and target text to proper word ids
    :param source_text: String that contains all the source text.
    :param target_text: String that contains all the target text.
    :param source_vocab_to_int: Dictionary to go from the source words to an id
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :return: A tuple of lists (source_id_text, target_id_text)
    """
    # TODO: Implement Function
    special_word = target_vocab_to_int['<EOS>']
    source = [[source_vocab_to_int[word] for word in sentence.split()] for sentence in source_text.split('\n')]
    target = [[target_vocab_to_int[word] for word in sentence.split()] +[special_word] for sentence in target_text.split('\n')]
    return source, target

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_text_to_ids(text_to_ids)

Tests Passed


### Preprocess all the data and save it
Running the code cell below will preprocess all the data and save it to file.

In [4]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
helper.preprocess_and_save_data(source_path, target_path, text_to_ids)

# Check Point
This is your first checkpoint. If you ever decide to come back to this notebook or have to restart the notebook, you can start from here. The preprocessed data has been saved to disk.

In [5]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import numpy as np
import helper
import problem_unittests as tests

(source_int_text, target_int_text), (source_vocab_to_int, target_vocab_to_int), _ = helper.load_preprocess()

### Check the Version of TensorFlow and Access to GPU
This will check to make sure you have the correct version of TensorFlow and access to a GPU

In [6]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from distutils.version import LooseVersion
import warnings
import tensorflow as tf
from tensorflow.python.layers.core import Dense

# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.1'), 'Please use TensorFlow version 1.1 or newer'
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.4.0
Default GPU Device: /device:GPU:0


## Build the Neural Network
You'll build the components necessary to build a Sequence-to-Sequence model by implementing the following functions below:
- `model_inputs`
- `process_decoder_input`
- `encoding_layer`
- `decoding_layer_train`
- `decoding_layer_infer`
- `decoding_layer`
- `seq2seq_model`

### Input
Implement the `model_inputs()` function to create TF Placeholders for the Neural Network. It should create the following placeholders:

- Input text placeholder named "input" using the TF Placeholder name parameter with rank 2.
- Targets placeholder with rank 2.
- Learning rate placeholder with rank 0.
- Keep probability placeholder named "keep_prob" using the TF Placeholder name parameter with rank 0.
- Target sequence length placeholder named "target_sequence_length" with rank 1
- Max target sequence length tensor named "max_target_len" getting its value from applying tf.reduce_max on the target_sequence_length placeholder. Rank 0.
- Source sequence length placeholder named "source_sequence_length" with rank 1

Return the placeholders in the following the tuple (input, targets, learning rate, keep probability, target sequence length, max target sequence length, source sequence length)

In [7]:
def model_inputs():
    """
    Create TF Placeholders for input, targets, learning rate, and lengths of source and target sequences.
    :return: Tuple (input, targets, learning rate, keep probability, target sequence length,
    max target sequence length, source sequence length)
    """
    # TODO: Implement Function
    input = tf.placeholder(tf.int32, shape=[None,None],name='input')
    targets = tf.placeholder(tf.int32, shape=[None,None],name='targets')
    learning_rate = tf.placeholder(tf.float32, shape=[],name='learning_rate')
    keep_prob = tf.placeholder(tf.float32, shape=[], name='keep_prob')
    target_sequence_length = tf.placeholder(tf.int32, shape=[None], name='target_sequence_length')
    max_target_len = tf.reduce_max(target_sequence_length, name='max_target_len')
    source_sequence_length = tf.placeholder(tf.int32, shape=[None], name='source_sequence_length')
    return input, targets, learning_rate, keep_prob, target_sequence_length, max_target_len, source_sequence_length


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_model_inputs(model_inputs)

Tests Passed


### Process Decoder Input
Implement `process_decoder_input` by removing the last word id from each batch in `target_data` and concat the GO ID to the begining of each batch.

In [8]:
def process_decoder_input(target_data, target_vocab_to_int, batch_size):
    """
    Preprocess target data for encoding
    :param target_data: Target Placehoder
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :param batch_size: Batch Size
    :return: Preprocessed target data
    """
    # TODO: Implement Function
    ending = tf.strided_slice(target_data, [0, 0], [batch_size, -1], [1, 1])
    dec_input = tf.concat([tf.fill([batch_size, 1], target_vocab_to_int['<GO>']), ending], 1)
    return dec_input

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_process_encoding_input(process_decoder_input)

Tests Passed


### Encoding
Implement `encoding_layer()` to create a Encoder RNN layer:
 * Embed the encoder input using [`tf.contrib.layers.embed_sequence`](https://www.tensorflow.org/api_docs/python/tf/contrib/layers/embed_sequence)
 * Construct a [stacked](https://github.com/tensorflow/tensorflow/blob/6947f65a374ebf29e74bb71e36fd82760056d82c/tensorflow/docs_src/tutorials/recurrent.md#stacking-multiple-lstms) [`tf.contrib.rnn.LSTMCell`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/LSTMCell) wrapped in a [`tf.contrib.rnn.DropoutWrapper`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/DropoutWrapper)
 * Pass cell and embedded input to [`tf.nn.dynamic_rnn()`](https://www.tensorflow.org/api_docs/python/tf/nn/dynamic_rnn)

In [9]:
from imp import reload
reload(tests)

def encoding_layer(rnn_inputs, rnn_size, num_layers, keep_prob, 
                   source_sequence_length, source_vocab_size, 
                   encoding_embedding_size):
    """
    Create encoding layer
    :param rnn_inputs: Inputs for the RNN
    :param rnn_size: RNN Size
    :param num_layers: Number of layers
    :param keep_prob: Dropout keep probability
    :param source_sequence_length: a list of the lengths of each sequence in the batch
    :param source_vocab_size: vocabulary size of source data
    :param encoding_embedding_size: embedding size of source data
    :return: tuple (RNN output, RNN state)
    """
    # TODO: Implement Function
    enc_emb_input = tf.contrib.layers.embed_sequence(rnn_inputs, source_vocab_size, encoding_embedding_size)
    
    def make_cell(rnn_size):
        enc_cell = tf.contrib.rnn.LSTMCell(rnn_size, initializer=tf.random_normal_initializer(-0.1, 0.1, seed=2))
        return enc_cell
    
    enc_cell = tf.contrib.rnn.MultiRNNCell([make_cell(rnn_size) for _ in range(num_layers)])
    enc_cell = tf.contrib.rnn.DropoutWrapper(enc_cell, output_keep_prob=keep_prob)
    enc_output, enc_state = tf.nn.dynamic_rnn(enc_cell, enc_emb_input, sequence_length=source_sequence_length,dtype=tf.float32)
    
    return enc_output, enc_state

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_encoding_layer(encoding_layer)

Tests Passed


### Decoding - Training
Create a training decoding layer:
* Create a [`tf.contrib.seq2seq.TrainingHelper`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/TrainingHelper) 
* Create a [`tf.contrib.seq2seq.BasicDecoder`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/BasicDecoder)
* Obtain the decoder outputs from [`tf.contrib.seq2seq.dynamic_decode`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/dynamic_decode)

In [10]:

def decoding_layer_train(encoder_state, dec_cell, dec_embed_input, 
                         target_sequence_length, max_summary_length, 
                         output_layer, keep_prob):
    """
    Create a decoding layer for training
    :param encoder_state: Encoder State
    :param dec_cell: Decoder RNN Cell
    :param dec_embed_input: Decoder embedded input
    :param target_sequence_length: The lengths of each sequence in the target batch
    :param max_summary_length: The length of the longest sequence in the batch
    :param output_layer: Function to apply the output layer
    :param keep_prob: Dropout keep probability
    :return: BasicDecoderOutput containing training logits and sample_id
    """
    # TODO: Implement Function
    training_helper = tf.contrib.seq2seq.TrainingHelper(inputs=dec_embed_input, sequence_length=target_sequence_length,
                                                       time_major=False)
    training_decoder = tf.contrib.seq2seq.BasicDecoder(dec_cell, training_helper, encoder_state, output_layer)
    
    training_decoder_output = tf.contrib.seq2seq.dynamic_decode(training_decoder,impute_finished=True, 
                                                                maximum_iterations=max_summary_length)[0]
    
    
    return training_decoder_output



"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_decoding_layer_train(decoding_layer_train)

Tests Passed


### Decoding - Inference
Create inference decoder:
* Create a [`tf.contrib.seq2seq.GreedyEmbeddingHelper`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/GreedyEmbeddingHelper)
* Create a [`tf.contrib.seq2seq.BasicDecoder`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/BasicDecoder)
* Obtain the decoder outputs from [`tf.contrib.seq2seq.dynamic_decode`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/dynamic_decode)

In [11]:
def decoding_layer_infer(encoder_state, dec_cell, dec_embeddings, start_of_sequence_id,
                         end_of_sequence_id, max_target_sequence_length,
                         vocab_size, output_layer, batch_size, keep_prob):
    """
    Create a decoding layer for inference
    :param encoder_state: Encoder state
    :param dec_cell: Decoder RNN Cell
    :param dec_embeddings: Decoder embeddings
    :param start_of_sequence_id: GO ID
    :param end_of_sequence_id: EOS Id
    :param max_target_sequence_length: Maximum length of target sequences
    :param vocab_size: Size of decoder/target vocabulary
    :param decoding_scope: TenorFlow Variable Scope for decoding
    :param output_layer: Function to apply the output layer
    :param batch_size: Batch size
    :param keep_prob: Dropout keep probability
    :return: BasicDecoderOutput containing inference logits and sample_id
    """
    # TODO: Implement Function
    start_tokens = tf.tile(tf.constant([target_vocab_to_int['<GO>']], dtype=tf.int32), [batch_size], name='start_tokens')

    
    inference_helper = tf.contrib.seq2seq.GreedyEmbeddingHelper(dec_embeddings, start_tokens, target_vocab_to_int['<EOS>'])
    
    inference_dec = tf.contrib.seq2seq.BasicDecoder(dec_cell, inference_helper, encoder_state, output_layer)
    
    inference_dec_o = tf.contrib.seq2seq.dynamic_decode(inference_dec, impute_finished=True, 
                                                        maximum_iterations=max_target_sequence_length)[0]
    
    return inference_dec_o



"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_decoding_layer_infer(decoding_layer_infer)

Tests Passed


### Build the Decoding Layer
Implement `decoding_layer()` to create a Decoder RNN layer.

* Embed the target sequences
* Construct the decoder LSTM cell (just like you constructed the encoder cell above)
* Create an output layer to map the outputs of the decoder to the elements of our vocabulary
* Use the your `decoding_layer_train(encoder_state, dec_cell, dec_embed_input, target_sequence_length, max_target_sequence_length, output_layer, keep_prob)` function to get the training logits.
* Use your `decoding_layer_infer(encoder_state, dec_cell, dec_embeddings, start_of_sequence_id, end_of_sequence_id, max_target_sequence_length, vocab_size, output_layer, batch_size, keep_prob)` function to get the inference logits.

Note: You'll need to use [tf.variable_scope](https://www.tensorflow.org/api_docs/python/tf/variable_scope) to share variables between training and inference.

In [12]:
def decoding_layer(dec_input, encoder_state,
                   target_sequence_length, max_target_sequence_length,
                   rnn_size,
                   num_layers, target_vocab_to_int, target_vocab_size,
                   batch_size, keep_prob, decoding_embedding_size):
    """
    Create decoding layer
    :param dec_input: Decoder input
    :param encoder_state: Encoder state
    :param target_sequence_length: The lengths of each sequence in the target batch
    :param max_target_sequence_length: Maximum length of target sequences
    :param rnn_size: RNN Size
    :param num_layers: Number of layers
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :param target_vocab_size: Size of target vocabulary
    :param batch_size: The size of the batch
    :param keep_prob: Dropout keep probability
    :param decoding_embedding_size: Decoding embedding size
    :return: Tuple of (Training BasicDecoderOutput, Inference BasicDecoderOutput)
    """
    # TODO: Implement Function
    #dec_emb_input = tf.contrib.layers.embed_sequence(rnn_inputs, source_vocab_size, encoding_embedding_size)
    
    
    def make_cell(rnn_size):
        dec_cell = tf.contrib.rnn.LSTMCell(rnn_size, initializer=tf.random_normal_initializer(-0.1, 0.1, seed=2))
        dec_cell = tf.contrib.rnn.DropoutWrapper(dec_cell, output_keep_prob=keep_prob)
        return dec_cell
    
    dec_cell = tf.contrib.rnn.MultiRNNCell([make_cell(rnn_size) for _ in range(num_layers)])
    
       
    dec_emb = tf.Variable(tf.random_uniform([target_vocab_size, decoding_embedding_size]))
    dec_emb_inputs = tf.nn.embedding_lookup(dec_emb, dec_input)
    
    output_layer = Dense(target_vocab_size, kernel_initializer=tf.truncated_normal_initializer(mean=0.0,stddev=0.1))
    start_of_sequence_id = target_vocab_to_int['<GO>']
    end_of_sequence_id = target_vocab_to_int['<EOS>']
    with tf.variable_scope("decode"):
        train_dec_out= decoding_layer_train( encoder_state,dec_cell,dec_emb_inputs,target_sequence_length,max_target_sequence_length,
                                        output_layer,
                                        keep_prob)

            
    with tf.variable_scope("decode", reuse=True):
        infer_dec_out = decoding_layer_infer(encoder_state,dec_cell, dec_emb, start_of_sequence_id,end_of_sequence_id,
                                        max_target_sequence_length, target_vocab_size,output_layer,
                                        batch_size,keep_prob)
                                        
    return train_dec_out, infer_dec_out
   

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_decoding_layer(decoding_layer)

Tests Passed


### Build the Neural Network
Apply the functions you implemented above to:

- Encode the input using your `encoding_layer(rnn_inputs, rnn_size, num_layers, keep_prob,  source_sequence_length, source_vocab_size, encoding_embedding_size)`.
- Process target data using your `process_decoder_input(target_data, target_vocab_to_int, batch_size)` function.
- Decode the encoded input using your `decoding_layer(dec_input, enc_state, target_sequence_length, max_target_sentence_length, rnn_size, num_layers, target_vocab_to_int, target_vocab_size, batch_size, keep_prob, dec_embedding_size)` function.

In [13]:
def seq2seq_model(input_data, target_data, keep_prob, batch_size,
                  source_sequence_length, target_sequence_length,
                  max_target_sentence_length,
                  source_vocab_size, target_vocab_size,
                  enc_embedding_size, dec_embedding_size,
                  rnn_size, num_layers, target_vocab_to_int):
    """
    Build the Sequence-to-Sequence part of the neural network
    :param input_data: Input placeholder
    :param target_data: Target placeholder
    :param keep_prob: Dropout keep probability placeholder
    :param batch_size: Batch Size
    :param source_sequence_length: Sequence Lengths of source sequences in the batch
    :param target_sequence_length: Sequence Lengths of target sequences in the batch
    :param source_vocab_size: Source vocabulary size
    :param target_vocab_size: Target vocabulary size
    :param enc_embedding_size: Decoder embedding size
    :param dec_embedding_size: Encoder embedding size
    :param rnn_size: RNN Size
    :param num_layers: Number of layers
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :return: Tuple of (Training BasicDecoderOutput, Inference BasicDecoderOutput)
    """
    # TODO: Implement Function
    _, enc_state=encoding_layer(input_data, rnn_size, num_layers, keep_prob,  source_sequence_length
                   , source_vocab_size, enc_embedding_size)
    
    dec_input = process_decoder_input(target_data, target_vocab_to_int, batch_size)
    
    tr_dec_output, inf_dec_output=decoding_layer(dec_input, enc_state, target_sequence_length, max_target_sentence_length,
                   rnn_size, num_layers, target_vocab_to_int, 
                   target_vocab_size, batch_size, keep_prob, dec_embedding_size)
    
    return tr_dec_output, inf_dec_output


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_seq2seq_model(seq2seq_model)

Tests Passed


## Neural Network Training
### Hyperparameters
Tune the following parameters:

- Set `epochs` to the number of epochs.
- Set `batch_size` to the batch size.
- Set `rnn_size` to the size of the RNNs.
- Set `num_layers` to the number of layers.
- Set `encoding_embedding_size` to the size of the embedding for the encoder.
- Set `decoding_embedding_size` to the size of the embedding for the decoder.
- Set `learning_rate` to the learning rate.
- Set `keep_probability` to the Dropout keep probability
- Set `display_step` to state how many steps between each debug output statement

In [26]:
# Number of Epochs
epochs = 5
# Batch Size
batch_size = 128
# RNN Size
rnn_size = 128
# Number of Layers
num_layers = 2
# Embedding Size
encoding_embedding_size =227
decoding_embedding_size = 227
# Learning Rate
learning_rate = 0.005
# Dropout Keep Probability
keep_probability = 0.8
display_step = True

### Build the Graph
Build the graph using the neural network you implemented.

In [27]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
save_path = 'checkpoints/dev'
(source_int_text, target_int_text), (source_vocab_to_int, target_vocab_to_int), _ = helper.load_preprocess()
max_target_sentence_length = max([len(sentence) for sentence in source_int_text])

train_graph = tf.Graph()
with train_graph.as_default():
    input_data, targets, lr, keep_prob, target_sequence_length, max_target_sequence_length, source_sequence_length = model_inputs()

    #sequence_length = tf.placeholder_with_default(max_target_sentence_length, None, name='sequence_length')
    input_shape = tf.shape(input_data)

    train_logits, inference_logits = seq2seq_model(tf.reverse(input_data, [-1]),
                                                   targets,
                                                   keep_prob,
                                                   batch_size,
                                                   source_sequence_length,
                                                   target_sequence_length,
                                                   max_target_sequence_length,
                                                   len(source_vocab_to_int),
                                                   len(target_vocab_to_int),
                                                   encoding_embedding_size,
                                                   decoding_embedding_size,
                                                   rnn_size,
                                                   num_layers,
                                                   target_vocab_to_int)


    training_logits = tf.identity(train_logits.rnn_output, name='logits')
    inference_logits = tf.identity(inference_logits.sample_id, name='predictions')

    masks = tf.sequence_mask(target_sequence_length, max_target_sequence_length, dtype=tf.float32, name='masks')

    with tf.name_scope("optimization"):
        # Loss function
        cost = tf.contrib.seq2seq.sequence_loss(
            training_logits,
            targets,
            masks)

        # Optimizer
        optimizer = tf.train.AdamOptimizer(lr)

        # Gradient Clipping
        gradients = optimizer.compute_gradients(cost)
        capped_gradients = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients if grad is not None]
        train_op = optimizer.apply_gradients(capped_gradients)


Batch and pad the source and target sequences

In [28]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
def pad_sentence_batch(sentence_batch, pad_int):
    """Pad sentences with <PAD> so that each sentence of a batch has the same length"""
    max_sentence = max([len(sentence) for sentence in sentence_batch])
    return [sentence + [pad_int] * (max_sentence - len(sentence)) for sentence in sentence_batch]


def get_batches(sources, targets, batch_size, source_pad_int, target_pad_int):
    """Batch targets, sources, and the lengths of their sentences together"""
    for batch_i in range(0, len(sources)//batch_size):
        start_i = batch_i * batch_size

        # Slice the right amount for the batch
        sources_batch = sources[start_i:start_i + batch_size]
        targets_batch = targets[start_i:start_i + batch_size]

        # Pad
        pad_sources_batch = np.array(pad_sentence_batch(sources_batch, source_pad_int))
        pad_targets_batch = np.array(pad_sentence_batch(targets_batch, target_pad_int))

        # Need the lengths for the _lengths parameters
        pad_targets_lengths = []
        for target in pad_targets_batch:
            pad_targets_lengths.append(len(target))

        pad_source_lengths = []
        for source in pad_sources_batch:
            pad_source_lengths.append(len(source))

        yield pad_sources_batch, pad_targets_batch, pad_source_lengths, pad_targets_lengths


### Train
Train the neural network on the preprocessed data. If you have a hard time getting a good loss, check the forms to see if anyone is having the same problem.

In [29]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
def get_accuracy(target, logits):
    """
    Calculate accuracy
    """
    max_seq = max(target.shape[1], logits.shape[1])
    if max_seq - target.shape[1]:
        target = np.pad(
            target,
            [(0,0),(0,max_seq - target.shape[1])],
            'constant')
    if max_seq - logits.shape[1]:
        logits = np.pad(
            logits,
            [(0,0),(0,max_seq - logits.shape[1])],
            'constant')

    return np.mean(np.equal(target, logits))

# Split data to training and validation sets
train_source = source_int_text[batch_size:]
train_target = target_int_text[batch_size:]
valid_source = source_int_text[:batch_size]
valid_target = target_int_text[:batch_size]
(valid_sources_batch, valid_targets_batch, valid_sources_lengths, valid_targets_lengths ) = next(get_batches(valid_source,
                                                                                                             valid_target,
                                                                                                             batch_size,
                                                                                                             source_vocab_to_int['<PAD>'],
                                                                                                             target_vocab_to_int['<PAD>']))                                                                                                  
with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())

    for epoch_i in range(epochs):
        for batch_i, (source_batch, target_batch, sources_lengths, targets_lengths) in enumerate(
                get_batches(train_source, train_target, batch_size,
                            source_vocab_to_int['<PAD>'],
                            target_vocab_to_int['<PAD>'])):

            _, loss = sess.run(
                [train_op, cost],
                {input_data: source_batch,
                 targets: target_batch,
                 lr: learning_rate,
                 target_sequence_length: targets_lengths,
                 source_sequence_length: sources_lengths,
                 keep_prob: keep_probability})


            if batch_i % display_step == 0 and batch_i > 0:


                batch_train_logits = sess.run(
                    inference_logits,
                    {input_data: source_batch,
                     source_sequence_length: sources_lengths,
                     target_sequence_length: targets_lengths,
                     keep_prob: 1.0})


                batch_valid_logits = sess.run(
                    inference_logits,
                    {input_data: valid_sources_batch,
                     source_sequence_length: valid_sources_lengths,
                     target_sequence_length: valid_targets_lengths,
                     keep_prob: 1.0})

                train_acc = get_accuracy(target_batch, batch_train_logits)

                valid_acc = get_accuracy(valid_targets_batch, batch_valid_logits)

                print('Epoch {:>3} Batch {:>4}/{} - Train Accuracy: {:>6.4f}, Validation Accuracy: {:>6.4f}, Loss: {:>6.4f}'
                      .format(epoch_i, batch_i, len(source_int_text) // batch_size, train_acc, valid_acc, loss))

    # Save Model
    saver = tf.train.Saver()
    saver.save(sess, save_path)
    print('Model Trained and Saved')

Epoch   0 Batch    1/1077 - Train Accuracy: 0.0137, Validation Accuracy: 0.0075, Loss: 5.8684
Epoch   0 Batch    2/1077 - Train Accuracy: 0.0103, Validation Accuracy: 0.0103, Loss: 5.8449
Epoch   0 Batch    3/1077 - Train Accuracy: 0.0141, Validation Accuracy: 0.0110, Loss: 5.8177
Epoch   0 Batch    4/1077 - Train Accuracy: 0.0156, Validation Accuracy: 0.0099, Loss: 5.7854
Epoch   0 Batch    5/1077 - Train Accuracy: 0.0293, Validation Accuracy: 0.0256, Loss: 5.7582
Epoch   0 Batch    6/1077 - Train Accuracy: 0.0629, Validation Accuracy: 0.0788, Loss: 5.7197
Epoch   0 Batch    7/1077 - Train Accuracy: 0.0754, Validation Accuracy: 0.0941, Loss: 5.6856
Epoch   0 Batch    8/1077 - Train Accuracy: 0.1680, Validation Accuracy: 0.2088, Loss: 5.6456
Epoch   0 Batch    9/1077 - Train Accuracy: 0.2328, Validation Accuracy: 0.3001, Loss: 5.6012
Epoch   0 Batch   10/1077 - Train Accuracy: 0.2019, Validation Accuracy: 0.3072, Loss: 5.5552
Epoch   0 Batch   11/1077 - Train Accuracy: 0.2604, Validati

Epoch   0 Batch   89/1077 - Train Accuracy: 0.3441, Validation Accuracy: 0.4006, Loss: 2.9944
Epoch   0 Batch   90/1077 - Train Accuracy: 0.3422, Validation Accuracy: 0.4002, Loss: 3.0078
Epoch   0 Batch   91/1077 - Train Accuracy: 0.3955, Validation Accuracy: 0.3949, Loss: 2.7525
Epoch   0 Batch   92/1077 - Train Accuracy: 0.3757, Validation Accuracy: 0.4059, Loss: 2.9030
Epoch   0 Batch   93/1077 - Train Accuracy: 0.3457, Validation Accuracy: 0.4016, Loss: 2.9582
Epoch   0 Batch   94/1077 - Train Accuracy: 0.3582, Validation Accuracy: 0.4055, Loss: 2.8827
Epoch   0 Batch   95/1077 - Train Accuracy: 0.3858, Validation Accuracy: 0.4055, Loss: 2.8168
Epoch   0 Batch   96/1077 - Train Accuracy: 0.3555, Validation Accuracy: 0.4055, Loss: 2.9029
Epoch   0 Batch   97/1077 - Train Accuracy: 0.3629, Validation Accuracy: 0.4077, Loss: 2.8637
Epoch   0 Batch   98/1077 - Train Accuracy: 0.3977, Validation Accuracy: 0.4006, Loss: 2.7207
Epoch   0 Batch   99/1077 - Train Accuracy: 0.3457, Validati

Epoch   0 Batch  178/1077 - Train Accuracy: 0.4492, Validation Accuracy: 0.5018, Loss: 2.2902
Epoch   0 Batch  179/1077 - Train Accuracy: 0.4420, Validation Accuracy: 0.5046, Loss: 2.3570
Epoch   0 Batch  180/1077 - Train Accuracy: 0.4367, Validation Accuracy: 0.4940, Loss: 2.2818
Epoch   0 Batch  181/1077 - Train Accuracy: 0.4516, Validation Accuracy: 0.4982, Loss: 2.2554
Epoch   0 Batch  182/1077 - Train Accuracy: 0.4550, Validation Accuracy: 0.4961, Loss: 2.2336
Epoch   0 Batch  183/1077 - Train Accuracy: 0.4309, Validation Accuracy: 0.4975, Loss: 2.3689
Epoch   0 Batch  184/1077 - Train Accuracy: 0.4684, Validation Accuracy: 0.4982, Loss: 2.2400
Epoch   0 Batch  185/1077 - Train Accuracy: 0.4324, Validation Accuracy: 0.4812, Loss: 2.2523
Epoch   0 Batch  186/1077 - Train Accuracy: 0.4317, Validation Accuracy: 0.4808, Loss: 2.3125
Epoch   0 Batch  187/1077 - Train Accuracy: 0.4348, Validation Accuracy: 0.5000, Loss: 2.3142
Epoch   0 Batch  188/1077 - Train Accuracy: 0.4480, Validati

Epoch   0 Batch  267/1077 - Train Accuracy: 0.4766, Validation Accuracy: 0.5018, Loss: 2.0137
Epoch   0 Batch  268/1077 - Train Accuracy: 0.4867, Validation Accuracy: 0.5156, Loss: 2.0724
Epoch   0 Batch  269/1077 - Train Accuracy: 0.4605, Validation Accuracy: 0.5323, Loss: 2.1717
Epoch   0 Batch  270/1077 - Train Accuracy: 0.4535, Validation Accuracy: 0.5312, Loss: 2.1469
Epoch   0 Batch  271/1077 - Train Accuracy: 0.4832, Validation Accuracy: 0.5341, Loss: 2.0284
Epoch   0 Batch  272/1077 - Train Accuracy: 0.4795, Validation Accuracy: 0.5206, Loss: 1.9886
Epoch   0 Batch  273/1077 - Train Accuracy: 0.4676, Validation Accuracy: 0.5124, Loss: 2.1310
Epoch   0 Batch  274/1077 - Train Accuracy: 0.4825, Validation Accuracy: 0.5238, Loss: 2.0449
Epoch   0 Batch  275/1077 - Train Accuracy: 0.5086, Validation Accuracy: 0.5281, Loss: 1.9220
Epoch   0 Batch  276/1077 - Train Accuracy: 0.4590, Validation Accuracy: 0.5284, Loss: 2.0996
Epoch   0 Batch  277/1077 - Train Accuracy: 0.5045, Validati

Epoch   0 Batch  355/1077 - Train Accuracy: 0.5476, Validation Accuracy: 0.5707, Loss: 1.6809
Epoch   0 Batch  356/1077 - Train Accuracy: 0.5457, Validation Accuracy: 0.5661, Loss: 1.6680
Epoch   0 Batch  357/1077 - Train Accuracy: 0.5848, Validation Accuracy: 0.5849, Loss: 1.6192
Epoch   0 Batch  358/1077 - Train Accuracy: 0.5317, Validation Accuracy: 0.5792, Loss: 1.7044
Epoch   0 Batch  359/1077 - Train Accuracy: 0.5453, Validation Accuracy: 0.5778, Loss: 1.6717
Epoch   0 Batch  360/1077 - Train Accuracy: 0.5441, Validation Accuracy: 0.5732, Loss: 1.6799
Epoch   0 Batch  361/1077 - Train Accuracy: 0.5530, Validation Accuracy: 0.5749, Loss: 1.6876
Epoch   0 Batch  362/1077 - Train Accuracy: 0.5595, Validation Accuracy: 0.5763, Loss: 1.6234
Epoch   0 Batch  363/1077 - Train Accuracy: 0.5133, Validation Accuracy: 0.5600, Loss: 1.7364
Epoch   0 Batch  364/1077 - Train Accuracy: 0.5363, Validation Accuracy: 0.5664, Loss: 1.7382
Epoch   0 Batch  365/1077 - Train Accuracy: 0.5188, Validati

...

Epoch   4 Batch  648/1077 - Train Accuracy: 0.9453, Validation Accuracy: 0.9286, Loss: 0.0648
Epoch   4 Batch  649/1077 - Train Accuracy: 0.9375, Validation Accuracy: 0.9247, Loss: 0.0728
Epoch   4 Batch  650/1077 - Train Accuracy: 0.9461, Validation Accuracy: 0.9247, Loss: 0.0636
Epoch   4 Batch  651/1077 - Train Accuracy: 0.9546, Validation Accuracy: 0.9368, Loss: 0.0673
Epoch   4 Batch  652/1077 - Train Accuracy: 0.9465, Validation Accuracy: 0.9375, Loss: 0.0752
Epoch   4 Batch  653/1077 - Train Accuracy: 0.9480, Validation Accuracy: 0.9386, Loss: 0.1045
Epoch   4 Batch  654/1077 - Train Accuracy: 0.9520, Validation Accuracy: 0.9371, Loss: 0.0618
Epoch   4 Batch  655/1077 - Train Accuracy: 0.8973, Validation Accuracy: 0.9325, Loss: 0.0830
Epoch   4 Batch  656/1077 - Train Accuracy: 0.9480, Validation Accuracy: 0.9325, Loss: 0.0681
Epoch   4 Batch  657/1077 - Train Accuracy: 0.9519, Validation Accuracy: 0.9322, Loss: 0.0718
Epoch   4 Batch  658/1077 - Train Accuracy: 0.9565, Validati

Epoch   4 Batch  736/1077 - Train Accuracy: 0.9338, Validation Accuracy: 0.9308, Loss: 0.0711
Epoch   4 Batch  737/1077 - Train Accuracy: 0.9566, Validation Accuracy: 0.9194, Loss: 0.0730
Epoch   4 Batch  738/1077 - Train Accuracy: 0.9421, Validation Accuracy: 0.9364, Loss: 0.0518
Epoch   4 Batch  739/1077 - Train Accuracy: 0.9348, Validation Accuracy: 0.9411, Loss: 0.0725
Epoch   4 Batch  740/1077 - Train Accuracy: 0.9488, Validation Accuracy: 0.9286, Loss: 0.0525
Epoch   4 Batch  741/1077 - Train Accuracy: 0.9383, Validation Accuracy: 0.9293, Loss: 0.0833
Epoch   4 Batch  742/1077 - Train Accuracy: 0.9414, Validation Accuracy: 0.9297, Loss: 0.0652
Epoch   4 Batch  743/1077 - Train Accuracy: 0.9445, Validation Accuracy: 0.9265, Loss: 0.0747
Epoch   4 Batch  744/1077 - Train Accuracy: 0.9405, Validation Accuracy: 0.9201, Loss: 0.0640
Epoch   4 Batch  745/1077 - Train Accuracy: 0.9406, Validation Accuracy: 0.9130, Loss: 0.0829
Epoch   4 Batch  746/1077 - Train Accuracy: 0.9508, Validati

Epoch   4 Batch  824/1077 - Train Accuracy: 0.9230, Validation Accuracy: 0.9396, Loss: 0.0796
Epoch   4 Batch  825/1077 - Train Accuracy: 0.9492, Validation Accuracy: 0.9421, Loss: 0.0596
Epoch   4 Batch  826/1077 - Train Accuracy: 0.9260, Validation Accuracy: 0.9414, Loss: 0.0588
Epoch   4 Batch  827/1077 - Train Accuracy: 0.9243, Validation Accuracy: 0.9425, Loss: 0.0731
Epoch   4 Batch  828/1077 - Train Accuracy: 0.9516, Validation Accuracy: 0.9425, Loss: 0.0517
Epoch   4 Batch  829/1077 - Train Accuracy: 0.9324, Validation Accuracy: 0.9393, Loss: 0.0748
Epoch   4 Batch  830/1077 - Train Accuracy: 0.9152, Validation Accuracy: 0.9396, Loss: 0.0755
Epoch   4 Batch  831/1077 - Train Accuracy: 0.9000, Validation Accuracy: 0.9350, Loss: 0.0654
Epoch   4 Batch  832/1077 - Train Accuracy: 0.9344, Validation Accuracy: 0.9290, Loss: 0.0614
Epoch   4 Batch  833/1077 - Train Accuracy: 0.9477, Validation Accuracy: 0.9279, Loss: 0.0694
Epoch   4 Batch  834/1077 - Train Accuracy: 0.9460, Validati

Epoch   4 Batch  913/1077 - Train Accuracy: 0.9293, Validation Accuracy: 0.9339, Loss: 0.0960
Epoch   4 Batch  914/1077 - Train Accuracy: 0.9453, Validation Accuracy: 0.9283, Loss: 0.0859
Epoch   4 Batch  915/1077 - Train Accuracy: 0.9046, Validation Accuracy: 0.9237, Loss: 0.0583
Epoch   4 Batch  916/1077 - Train Accuracy: 0.9492, Validation Accuracy: 0.9229, Loss: 0.0746
Epoch   4 Batch  917/1077 - Train Accuracy: 0.9387, Validation Accuracy: 0.9379, Loss: 0.0605
Epoch   4 Batch  918/1077 - Train Accuracy: 0.9650, Validation Accuracy: 0.9371, Loss: 0.0588
Epoch   4 Batch  919/1077 - Train Accuracy: 0.9564, Validation Accuracy: 0.9421, Loss: 0.0525
Epoch   4 Batch  920/1077 - Train Accuracy: 0.9496, Validation Accuracy: 0.9361, Loss: 0.0551
Epoch   4 Batch  921/1077 - Train Accuracy: 0.9348, Validation Accuracy: 0.9237, Loss: 0.0782
Epoch   4 Batch  922/1077 - Train Accuracy: 0.9479, Validation Accuracy: 0.9283, Loss: 0.0674
Epoch   4 Batch  923/1077 - Train Accuracy: 0.9650, Validati

Epoch   4 Batch 1001/1077 - Train Accuracy: 0.9574, Validation Accuracy: 0.9286, Loss: 0.0475
Epoch   4 Batch 1002/1077 - Train Accuracy: 0.9660, Validation Accuracy: 0.9325, Loss: 0.0420
Epoch   4 Batch 1003/1077 - Train Accuracy: 0.9634, Validation Accuracy: 0.9329, Loss: 0.0560
Epoch   4 Batch 1004/1077 - Train Accuracy: 0.9586, Validation Accuracy: 0.9354, Loss: 0.0748
Epoch   4 Batch 1005/1077 - Train Accuracy: 0.9426, Validation Accuracy: 0.9357, Loss: 0.0606
Epoch   4 Batch 1006/1077 - Train Accuracy: 0.9375, Validation Accuracy: 0.9364, Loss: 0.0501
Epoch   4 Batch 1007/1077 - Train Accuracy: 0.9609, Validation Accuracy: 0.9396, Loss: 0.0548
Epoch   4 Batch 1008/1077 - Train Accuracy: 0.9461, Validation Accuracy: 0.9343, Loss: 0.0910
Epoch   4 Batch 1009/1077 - Train Accuracy: 0.9668, Validation Accuracy: 0.9343, Loss: 0.0502
Epoch   4 Batch 1010/1077 - Train Accuracy: 0.9773, Validation Accuracy: 0.9347, Loss: 0.0519
Epoch   4 Batch 1011/1077 - Train Accuracy: 0.9512, Validati

### Save Parameters
Save the `batch_size` and `save_path` parameters for inference.

In [18]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Save parameters for checkpoint
helper.save_params(save_path)

# Checkpoint

In [19]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import tensorflow as tf
import numpy as np
import helper
import problem_unittests as tests

_, (source_vocab_to_int, target_vocab_to_int), (source_int_to_vocab, target_int_to_vocab) = helper.load_preprocess()
load_path = helper.load_params()

## Sentence to Sequence
To feed a sentence into the model for translation, you first need to preprocess it.  Implement the function `sentence_to_seq()` to preprocess new sentences.

- Convert the sentence to lowercase
- Convert words into ids using `vocab_to_int`
 - Convert words not in the vocabulary, to the `<UNK>` word id.

In [20]:
def sentence_to_seq(sentence, vocab_to_int):
    """
    Convert a sentence to a sequence of ids
    :param sentence: String
    :param vocab_to_int: Dictionary to go from the words to an id
    :return: List of word ids
    """
    # TODO: Implement Function
    unk = vocab_to_int['<UNK>']
    lower_unk=[vocab_to_int.get(word, unk) for word in sentence.lower().split()]
    return lower_unk


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_sentence_to_seq(sentence_to_seq)

Tests Passed


## Translate
This will translate `translate_sentence` from English to French.

In [21]:
translate_sentence = 'he saw a old yellow truck .'


"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
translate_sentence = sentence_to_seq(translate_sentence, source_vocab_to_int)

loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.train.import_meta_graph(load_path + '.meta')
    loader.restore(sess, load_path)

    input_data = loaded_graph.get_tensor_by_name('input:0')
    logits = loaded_graph.get_tensor_by_name('predictions:0')
    target_sequence_length = loaded_graph.get_tensor_by_name('target_sequence_length:0')
    source_sequence_length = loaded_graph.get_tensor_by_name('source_sequence_length:0')
    keep_prob = loaded_graph.get_tensor_by_name('keep_prob:0')

    translate_logits = sess.run(logits, {input_data: [translate_sentence]*batch_size,
                                         target_sequence_length: [len(translate_sentence)*2]*batch_size,
                                         source_sequence_length: [len(translate_sentence)]*batch_size,
                                         keep_prob: 1.0})[0]

print('Input')
print('  Word Ids:      {}'.format([i for i in translate_sentence]))
print('  English Words: {}'.format([source_int_to_vocab[i] for i in translate_sentence]))

print('\nPrediction')
print('  Word Ids:      {}'.format([i for i in translate_logits]))
print('  French Words: {}'.format(" ".join([target_int_to_vocab[i] for i in translate_logits])))


INFO:tensorflow:Restoring parameters from checkpoints/dev
Input
  Word Ids:      [93, 189, 210, 175, 211, 103, 193]
  English Words: ['he', 'saw', 'a', 'old', 'yellow', 'truck', '.']

Prediction
  Word Ids:      [273, 353, 28, 186, 138, 41, 174, 1]
  French Words: il a vu un vieux camion . <EOS>


## Imperfect Translation
You might notice that some sentences translate better than others.  Since the dataset you're using only has a vocabulary of 227 English words of the thousands that you use, you're only going to see good results using these words.  For this project, you don't need a perfect translation. However, if you want to create a better translation model, you'll need better data.

You can train on the [WMT10 French-English corpus](http://www.statmt.org/wmt10/training-giga-fren.tar).  This dataset has more vocabulary and richer in topics discussed.  However, this will take you days to train, so make sure you've a GPU and the neural network is performing well on dataset we provided.  Just make sure you play with the WMT10 corpus after you've submitted this project.
## Submitting This Project
When submitting this project, make sure to run all the cells before saving the notebook. Save the notebook file as "dlnd_language_translation.ipynb" and save it as a HTML file under "File" -> "Download as". Include the "helper.py" and "problem_unittests.py" files in your submission.